# Kosmic Lab Interactive Tutorial

**Version**: 1.1.0  
**Last Updated**: 2025-11-15

Welcome to the **Kosmic Lab Interactive Tutorial**! This notebook will guide you through all major features of Kosmic Lab with executable code examples.

---

## Table of Contents

1. [Setup & Imports](#1.-Setup-&-Imports)
2. [Core Utilities](#2.-Core-Utilities)
3. [K-Index Metrics](#3.-K-Index-Metrics)
4. [K-Lag Analysis](#4.-K-Lag-Analysis)
5. [K-Codex System](#5.-K-Codex-System)
6. [Bioelectric Simulation](#6.-Bioelectric-Simulation)
7. [FRE Universe Simulation](#7.-FRE-Universe-Simulation)
8. [Complete Experiment Example](#8.-Complete-Experiment-Example)
9. [Visualization](#9.-Visualization)
10. [Next Steps](#10.-Next-Steps)

---

## Prerequisites

Make sure you have installed Kosmic Lab:

```bash
poetry install
```

Or if running this notebook, ensure the kernel has access to the Kosmic Lab environment.

## 1. Setup & Imports

Let's start by importing all the libraries we'll need:

In [ ]:
# Core libraries
import numpy as np
import matplotlib.pyplot as plt
import json
from pathlib import Path

# Kosmic Lab core utilities
from core.utils import bootstrap_confidence_interval, infer_git_sha, safe_divide
from core.logging_config import setup_logging, get_logger
from core.kcodex import KCodexWriter

# K-Index metrics
from fre.metrics.k_index import k_index, bootstrap_k_ci
from fre.metrics.k_lag import k_lag, verify_causal_direction

# Simulations
from core.bioelectric import BioelectricGrid
from fre.universe import UniverseSimulator

# Setup logging
setup_logging(level="INFO")
logger = get_logger(__name__)

# Set random seed for reproducibility
SEED = 42
rng = np.random.default_rng(SEED)

# Configure matplotlib for inline display
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("✅ All imports successful!")
print(f"📍 Git SHA: {infer_git_sha()}")
print(f"🎲 Random seed: {SEED}")

## 2. Core Utilities

Kosmic Lab provides several core utilities that are used throughout the framework.

### Bootstrap Confidence Intervals

Bootstrap resampling provides robust confidence intervals for any statistic:

In [ ]:
# Generate sample data
data = rng.normal(loc=10, scale=2, size=100)

# Compute bootstrap CI for the mean
ci_lower, ci_upper = bootstrap_confidence_interval(
    data,
    statistic=np.mean,
    n_bootstrap=1000,
    confidence_level=0.95,
    seed=SEED
)

print(f"Data mean: {np.mean(data):.4f}")
print(f"95% CI: [{ci_lower:.4f}, {ci_upper:.4f}]")
print(f"CI width: {ci_upper - ci_lower:.4f}")

# Try with a different statistic (median)
ci_lower_med, ci_upper_med = bootstrap_confidence_interval(
    data,
    statistic=np.median,
    n_bootstrap=1000,
    confidence_level=0.95,
    seed=SEED
)

print(f"\nData median: {np.median(data):.4f}")
print(f"95% CI: [{ci_lower_med:.4f}, {ci_upper_med:.4f}]")

### Safe Division

Avoid division by zero errors with safe defaults:

In [ ]:
# Normal division
result1 = safe_divide(10, 2, default=0.0)
print(f"10 / 2 = {result1}")

# Division by zero (returns default)
result2 = safe_divide(10, 0, default=0.0)
print(f"10 / 0 = {result2} (default)")

# Custom default
result3 = safe_divide(10, 0, default=np.nan)
print(f"10 / 0 = {result3} (NaN default)")

## 3. K-Index Metrics

K-Index is our primary metric for measuring observer-observed coherence.

### Basic K-Index Computation

In [ ]:
# Generate synthetic data
n_samples = 1000
observed = rng.random(n_samples)
actual = rng.random(n_samples)

# Compute K-Index
k = k_index(observed, actual)

print(f"K-Index: {k:.4f}")
print(f"\nInterpretation:")
if k > 0.8:
    print("  ✓ Excellent coherence (K > 0.8)")
elif k > 0.6:
    print("  ✓ Good coherence (0.6 < K ≤ 0.8)")
elif k > 0.4:
    print("  ⚠ Moderate coherence (0.4 < K ≤ 0.6)")
else:
    print("  ⚠ Low coherence (K ≤ 0.4)")

### K-Index with Confidence Intervals

For robust analysis, always compute confidence intervals:

In [ ]:
# Generate correlated data for better K-Index
actual_corr = rng.random(n_samples)
noise = rng.normal(0, 0.1, n_samples)
observed_corr = actual_corr + noise

# Compute K-Index with bootstrap CI
k_val, ci_lower, ci_upper = bootstrap_k_ci(
    observed_corr,
    actual_corr,
    n_bootstrap=1000,
    confidence_level=0.95,
    seed=SEED
)

print(f"K-Index: {k_val:.4f}")
print(f"95% CI: [{ci_lower:.4f}, {ci_upper:.4f}]")
print(f"CI width: {ci_upper - ci_lower:.4f}")
print(f"\n✓ Statistically significant: {ci_lower > 0}")

### Visualize K-Index Distribution

In [ ]:
# Generate multiple K-Index samples for visualization
k_samples = []
for i in range(100):
    sample_rng = np.random.default_rng(SEED + i)
    obs_sample = sample_rng.random(100)
    act_sample = sample_rng.random(100)
    k_samples.append(k_index(obs_sample, act_sample))

# Plot histogram
plt.figure(figsize=(10, 6))
plt.hist(k_samples, bins=20, alpha=0.7, edgecolor='black')
plt.axvline(np.mean(k_samples), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(k_samples):.4f}')
plt.xlabel('K-Index', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.title('K-Index Distribution (100 samples, N=100 each)', fontsize=16)
plt.legend(fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Mean K-Index: {np.mean(k_samples):.4f}")
print(f"Std K-Index: {np.std(k_samples):.4f}")

## 4. K-Lag Analysis

K-Lag helps identify temporal relationships between observed and actual data.

### Basic K-Lag

In [ ]:
# Generate time series with lag
n_timesteps = 500
actual_ts = np.sin(np.linspace(0, 4*np.pi, n_timesteps)) + rng.normal(0, 0.1, n_timesteps)

# Create observed with 5-timestep lag
lag_amount = 5
observed_ts = np.roll(actual_ts, lag_amount) + rng.normal(0, 0.05, n_timesteps)

# Perform K-Lag analysis
results = k_lag(
    observed_ts,
    actual_ts,
    max_lag=20,
    seed=SEED
)

print(f"Best lag: {results['best_lag']} timesteps")
print(f"K at best lag: {results['k_at_best_lag']:.4f}")
print(f"K at zero lag: {results['k_at_zero_lag']:.4f}")
print(f"\nImprovement: {results['k_at_best_lag'] - results['k_at_zero_lag']:.4f}")
print(f"✓ Detected lag matches true lag: {results['best_lag'] == lag_amount}")

### Visualize K-Lag Results

In [ ]:
# Plot K-Index vs lag
plt.figure(figsize=(12, 5))

# Subplot 1: K-Index vs Lag
plt.subplot(1, 2, 1)
plt.plot(results['lags'], results['k_values'], 'o-', linewidth=2, markersize=6)
plt.axvline(results['best_lag'], color='red', linestyle='--', linewidth=2, label=f"Best lag: {results['best_lag']}")
plt.axvline(lag_amount, color='green', linestyle='--', linewidth=2, label=f"True lag: {lag_amount}")
plt.xlabel('Lag (timesteps)', fontsize=14)
plt.ylabel('K-Index', fontsize=14)
plt.title('K-Index vs Lag', fontsize=16)
plt.legend(fontsize=12)
plt.grid(alpha=0.3)

# Subplot 2: Time series comparison
plt.subplot(1, 2, 2)
t = np.arange(100)  # Plot first 100 points for clarity
plt.plot(t, actual_ts[:100], 'b-', linewidth=2, label='Actual', alpha=0.7)
plt.plot(t, observed_ts[:100], 'r--', linewidth=2, label='Observed (lagged)', alpha=0.7)
plt.xlabel('Time', fontsize=14)
plt.ylabel('Value', fontsize=14)
plt.title('Time Series Comparison', fontsize=16)
plt.legend(fontsize=12)
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

### Verify Causal Direction

In [ ]:
# Check if observed follows actual (correct direction)
is_correct, forward_k, backward_k = verify_causal_direction(
    observed_ts,
    actual_ts,
    max_lag=20
)

print(f"Forward K (observed follows actual): {forward_k:.4f}")
print(f"Backward K (actual follows observed): {backward_k:.4f}")
print(f"\nCausal direction correct: {is_correct}")

if is_correct:
    print("✓ Observed follows actual (expected direction)")
else:
    print("⚠ Possible reversed causality detected!")

## 5. K-Codex System

K-Codex provides perfect reproducibility by logging all experiment parameters and results.

### Basic K-Codex Usage

In [ ]:
# Create K-Codex writer (using temporary file for this tutorial)
kcodex_path = Path("/tmp/tutorial_kcodex.json")
kcodex = KCodexWriter(str(kcodex_path))

# Run a simple experiment
experiment_seed = 123
experiment_rng = np.random.default_rng(experiment_seed)

obs_exp = experiment_rng.random(1000)
act_exp = experiment_rng.random(1000)
k_exp = k_index(obs_exp, act_exp)

# Log to K-Codex
kcodex.log_experiment(
    experiment_name="tutorial_experiment",
    params={
        "n_samples": 1000,
        "distribution": "uniform",
        "correlation_method": "pearsonr"
    },
    metrics={
        "k_index": k_exp,
        "mean_observed": float(np.mean(obs_exp)),
        "mean_actual": float(np.mean(act_exp))
    },
    seed=experiment_seed,
    extra_metadata={
        "notebook": "interactive_tutorial.ipynb",
        "purpose": "demonstration"
    }
)

print("✅ Experiment logged to K-Codex!")
print(f"📄 K-Codex file: {kcodex_path}")

### View K-Codex Entry

In [ ]:
# Load and display K-Codex entry
with open(kcodex_path) as f:
    kcodex_data = json.load(f)

print("📋 K-Codex Entry:")
print("="*60)
print(json.dumps(kcodex_data, indent=2))
print("="*60)

# Highlight key fields
print("\n🔑 Key Fields:")
print(f"  Run ID: {kcodex_data['run_id']}")
print(f"  Commit: {kcodex_data['commit']}")
print(f"  Config Hash: {kcodex_data['config_hash']}")
print(f"  Timestamp: {kcodex_data['timestamp']}")
print(f"  Seed: {kcodex_data['seed']}")
print(f"  K-Index: {kcodex_data['metrics']['k_index']:.6f}")

### Reproduce Experiment from K-Codex

In [ ]:
# Extract parameters from K-Codex
params_repro = kcodex_data['params']
seed_repro = kcodex_data['seed']

# Reproduce experiment
repro_rng = np.random.default_rng(seed_repro)
obs_repro = repro_rng.random(params_repro['n_samples'])
act_repro = repro_rng.random(params_repro['n_samples'])
k_repro = k_index(obs_repro, act_repro)

# Compare with original
k_original = kcodex_data['metrics']['k_index']

print("🔬 Reproducibility Test:")
print(f"  Original K-Index:    {k_original:.10f}")
print(f"  Reproduced K-Index:  {k_repro:.10f}")
print(f"  Difference:          {abs(k_original - k_repro):.2e}")

# Verify bit-for-bit reproducibility
if abs(k_original - k_repro) < 1e-10:
    print("\n✅ PERFECT REPRODUCIBILITY! (bit-for-bit identical)")
else:
    print("\n⚠ Warning: Results differ slightly")

## 6. Bioelectric Simulation

Simulate bioelectric phenomena on a 2D grid.

### Basic Bioelectric Grid

In [ ]:
# Create bioelectric grid
grid = BioelectricGrid(
    size=(20, 20),
    diffusion_rate=0.1,
    resting_potential=-70.0,
    seed=SEED
)

# Apply stimulation at center
center_x, center_y = 10, 10
grid.apply_stimulation(center_x, center_y, intensity=50.0)

print(f"Grid size: {grid.voltage_grid.shape}")
print(f"Resting potential: {grid.resting_potential} mV")
print(f"Initial avg voltage: {grid.get_average_voltage():.2f} mV")

### Simulate Bioelectric Propagation

In [ ]:
# Store snapshots at different timesteps
snapshots = [grid.voltage_grid.copy()]
snapshot_times = [0]

# Simulate 100 timesteps
for t in range(1, 101):
    grid.step()
    
    # Store snapshots at specific times
    if t in [10, 25, 50, 100]:
        snapshots.append(grid.voltage_grid.copy())
        snapshot_times.append(t)

print(f"Final avg voltage: {grid.get_average_voltage():.2f} mV")
print(f"Captured {len(snapshots)} snapshots at times: {snapshot_times}")

### Visualize Bioelectric Propagation

In [ ]:
# Plot snapshots
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, (snapshot, time) in enumerate(zip(snapshots, snapshot_times)):
    im = axes[idx].imshow(snapshot, cmap='RdBu_r', vmin=-70, vmax=30)
    axes[idx].set_title(f't = {time} steps', fontsize=14)
    axes[idx].set_xlabel('X', fontsize=12)
    axes[idx].set_ylabel('Y', fontsize=12)
    plt.colorbar(im, ax=axes[idx], label='Voltage (mV)')

# Hide unused subplot
axes[-1].axis('off')

plt.suptitle('Bioelectric Propagation Over Time', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

## 7. FRE Universe Simulation

Simulate the Free Research Environment (FRE) universe with consciousness dynamics.

### Basic Universe Simulation

In [ ]:
# Create universe simulator
simulator = UniverseSimulator(
    seed=SEED,
    n_timesteps=1000
)

# Run simulation
results = simulator.run(
    params={
        "consciousness": 0.5,
        "coherence": 0.7,
        "fep_learning_rate": 0.01
    }
)

print("🌌 Universe Simulation Results:")
print(f"  K-Index: {results['metrics']['k_index']:.4f}")
print(f"  Final state shape: {results['final_state'].shape if hasattr(results['final_state'], 'shape') else 'N/A'}")
print(f"  Trajectory length: {len(results['trajectory']) if 'trajectory' in results else 'N/A'}")

### Parameter Sweep

In [ ]:
# Sweep consciousness parameter
consciousness_values = np.linspace(0.1, 0.9, 5)
sweep_results = []

print("Running parameter sweep...")
for c_val in consciousness_values:
    sim = UniverseSimulator(seed=SEED, n_timesteps=500)
    result = sim.run({
        "consciousness": c_val,
        "coherence": 0.7,
        "fep_learning_rate": 0.01
    })
    
    sweep_results.append({
        "consciousness": c_val,
        "k_index": result['metrics']['k_index']
    })
    print(f"  consciousness={c_val:.2f} → K-Index={result['metrics']['k_index']:.4f}")

print("\n✅ Parameter sweep complete!")

### Visualize Parameter Sweep

In [ ]:
# Extract data for plotting
c_vals = [r['consciousness'] for r in sweep_results]
k_vals = [r['k_index'] for r in sweep_results]

# Plot
plt.figure(figsize=(10, 6))
plt.plot(c_vals, k_vals, 'o-', linewidth=2, markersize=10)
plt.xlabel('Consciousness Parameter', fontsize=14)
plt.ylabel('K-Index', fontsize=14)
plt.title('K-Index vs Consciousness Parameter', fontsize=16)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Find optimal consciousness
best_idx = np.argmax(k_vals)
print(f"\n🎯 Optimal consciousness: {c_vals[best_idx]:.2f}")
print(f"   Maximum K-Index: {k_vals[best_idx]:.4f}")

## 8. Complete Experiment Example

Let's put everything together in a complete experiment with K-Codex logging.

### Full Experimental Workflow

In [ ]:
# Setup
experiment_name = "complete_tutorial_experiment"
experiment_seed = 999
n_samples = 2000

logger.info(f"Starting {experiment_name}...")

# Create K-Codex writer
complete_kcodex_path = Path(f"/tmp/{experiment_name}_kcodex.json")
complete_kcodex = KCodexWriter(str(complete_kcodex_path))

# Generate data with known correlation
exp_rng = np.random.default_rng(experiment_seed)
actual_data = exp_rng.normal(0, 1, n_samples)
noise = exp_rng.normal(0, 0.2, n_samples)
observed_data = 0.8 * actual_data + 0.2 * noise  # Strong correlation

# Compute K-Index with CI
k_complete, ci_low, ci_high = bootstrap_k_ci(
    observed_data,
    actual_data,
    n_bootstrap=1000,
    confidence_level=0.95,
    seed=experiment_seed
)

logger.info(f"K-Index: {k_complete:.4f} [{ci_low:.4f}, {ci_high:.4f}]")

# Additional metrics
correlation = np.corrcoef(observed_data, actual_data)[0, 1]
rmse = np.sqrt(np.mean((observed_data - actual_data)**2))

# Log to K-Codex
complete_kcodex.log_experiment(
    experiment_name=experiment_name,
    params={
        "n_samples": n_samples,
        "correlation_strength": 0.8,
        "noise_std": 0.2,
        "n_bootstrap": 1000,
        "confidence_level": 0.95
    },
    metrics={
        "k_index": k_complete,
        "ci_lower": ci_low,
        "ci_upper": ci_high,
        "ci_width": ci_high - ci_low,
        "pearson_r": correlation,
        "rmse": rmse
    },
    seed=experiment_seed,
    estimators={
        "correlation_method": "scipy.stats.pearsonr",
        "bootstrap_method": "percentile"
    },
    extra_metadata={
        "notebook": "interactive_tutorial.ipynb",
        "section": "complete_experiment",
        "data_generation": "linear_mix_with_noise"
    }
)

logger.info("Experiment complete! Results logged to K-Codex.")

print("\n" + "="*60)
print("🎓 COMPLETE EXPERIMENT RESULTS")
print("="*60)
print(f"Experiment: {experiment_name}")
print(f"Samples: {n_samples}")
print(f"Seed: {experiment_seed}")
print(f"\nMetrics:")
print(f"  K-Index: {k_complete:.4f}")
print(f"  95% CI: [{ci_low:.4f}, {ci_high:.4f}]")
print(f"  Pearson r: {correlation:.4f}")
print(f"  RMSE: {rmse:.4f}")
print(f"\nK-Codex: {complete_kcodex_path}")
print("="*60)

## 9. Visualization

Visualize the complete experiment results.

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Scatter plot
axes[0, 0].scatter(actual_data[:500], observed_data[:500], alpha=0.5, s=20)
axes[0, 0].plot([actual_data.min(), actual_data.max()], 
                [actual_data.min(), actual_data.max()], 
                'r--', linewidth=2, label='Perfect correlation')
axes[0, 0].set_xlabel('Actual', fontsize=12)
axes[0, 0].set_ylabel('Observed', fontsize=12)
axes[0, 0].set_title('Observed vs Actual (first 500 samples)', fontsize=14)
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(alpha=0.3)

# Plot 2: Time series
t = np.arange(200)
axes[0, 1].plot(t, actual_data[:200], 'b-', linewidth=2, label='Actual', alpha=0.7)
axes[0, 1].plot(t, observed_data[:200], 'r--', linewidth=2, label='Observed', alpha=0.7)
axes[0, 1].set_xlabel('Sample', fontsize=12)
axes[0, 1].set_ylabel('Value', fontsize=12)
axes[0, 1].set_title('Time Series (first 200 samples)', fontsize=14)
axes[0, 1].legend(fontsize=10)
axes[0, 1].grid(alpha=0.3)

# Plot 3: Residuals
residuals = observed_data - actual_data
axes[1, 0].hist(residuals, bins=50, alpha=0.7, edgecolor='black')
axes[1, 0].axvline(0, color='red', linestyle='--', linewidth=2, label='Zero')
axes[1, 0].set_xlabel('Residual (Observed - Actual)', fontsize=12)
axes[1, 0].set_ylabel('Frequency', fontsize=12)
axes[1, 0].set_title(f'Residual Distribution (mean={np.mean(residuals):.4f})', fontsize=14)
axes[1, 0].legend(fontsize=10)
axes[1, 0].grid(alpha=0.3)

# Plot 4: Summary statistics
axes[1, 1].axis('off')
summary_text = f"""
EXPERIMENT SUMMARY
{'='*40}

Sample Size: {n_samples}
Seed: {experiment_seed}

K-Index: {k_complete:.4f}
95% CI: [{ci_low:.4f}, {ci_high:.4f}]
CI Width: {ci_high - ci_low:.4f}

Pearson r: {correlation:.4f}
RMSE: {rmse:.4f}

Mean Residual: {np.mean(residuals):.4f}
Std Residual: {np.std(residuals):.4f}

✅ Statistically Significant: {ci_low > 0}
✅ Reproducible: See K-Codex
"""
axes[1, 1].text(0.1, 0.5, summary_text, fontsize=11, family='monospace',
                verticalalignment='center')

plt.suptitle(f'{experiment_name.replace("_", " ").title()}', fontsize=16, y=1.00)
plt.tight_layout()
plt.show()

## 10. Next Steps

Congratulations! You've completed the Kosmic Lab interactive tutorial. 🎉

### What You've Learned

- ✅ Core utilities (bootstrap CI, safe division, git SHA)
- ✅ K-Index metrics with confidence intervals
- ✅ K-Lag temporal analysis
- ✅ K-Codex perfect reproducibility system
- ✅ Bioelectric grid simulation
- ✅ FRE universe simulation
- ✅ Complete experimental workflow
- ✅ Data visualization

### Explore More

1. **Examples Directory**
   - `examples/01_hello_kosmic.py` - Basic introduction
   - `examples/02_advanced_k_index.py` - Advanced K-Index usage
   - `examples/03_multi_universe.py` - Multi-universe simulations
   - `examples/04_bioelectric_rescue.py` - Bioelectric morphogenesis

2. **Documentation**
   - [QUICKSTART.md](../QUICKSTART.md) - 5-minute quick start
   - [API_REFERENCE.md](../API_REFERENCE.md) - API quick reference
   - [FAQ.md](../FAQ.md) - 50+ frequently asked questions
   - [Full Documentation](../docs/) - Complete technical docs

3. **Advanced Topics**
   - Performance profiling: `make profile`
   - Benchmarks: `make benchmarks`
   - Dashboard: `make dashboard`
   - Custom experiments with K-Codex

4. **Get Involved**
   - [CONTRIBUTING.md](../CONTRIBUTING.md) - Contribution guide
   - [GOVERNANCE.md](../GOVERNANCE.md) - Project governance
   - [GitHub Issues](https://github.com/Luminous-Dynamics/kosmic-lab/issues) - Report bugs or request features

### Try These Challenges

1. **Challenge 1**: Create your own experiment with custom parameters and log it to K-Codex
2. **Challenge 2**: Run a parameter sweep with 3 parameters and find the optimal combination
3. **Challenge 3**: Create a bioelectric simulation with multiple stimulation points
4. **Challenge 4**: Reproduce an experiment from K-Codex and verify bit-for-bit reproducibility
5. **Challenge 5**: Visualize a 3D parameter sweep (consciousness, coherence, learning rate)

### Resources

- **Documentation**: https://kosmic-lab.readthedocs.io (when available)
- **GitHub**: https://github.com/Luminous-Dynamics/kosmic-lab
- **Issues**: https://github.com/Luminous-Dynamics/kosmic-lab/issues
- **Discussions**: https://github.com/Luminous-Dynamics/kosmic-lab/discussions

---

**Happy experimenting! 🚀**

*Remember: All experiments are perfectly reproducible with K-Codex. Your discoveries are永 永永永.*